# 시사플러스 2019 서울맛집 지도화

## 개요

* 시사플러스에 게시된 [시사플러스 '2019 서울맛집'](http://m.sisaplusnews.com/news/articleView.html?idxno=22423) 기사의 맛집들을 지도 위에 표시하는 프로젝트입니다.
* 이 프로젝트는 [Chang-Joo Park](https://github.com/ChangJoo-Park)님께서 먼저 제안하시고 GeoJSON 사용법 등을 보여주셨습니다. 감사합니다. (_ _)
* 정확한 위치를 지도에 표시하기 보다는 카테고리별 맛집의 분포 등을 재미삼아 보고자 표시하였습니다.

## 정제

* 기초 데이터는 [시사플러스 '2019 서울맛집'](http://m.sisaplusnews.com/news/articleView.html?idxno=22423)에서 가져왔습니다.
* 전체 데이터 중 `1. 평양냉면`부터 `45. 돼지고기`까지 가져왔으며, 이후 데이터는 손을 많이 탈 것 같아 따로 정제하지 않았습니다.
* 기초 데이터는 `동 이름` (` - ` `역 이름`) ` - ` `음식점 이름` (` - ` `기타`) 형태로 정리하였으며, 가게명만 있는 경우 모두 제외하였습니다.
  * (`45. 돼지고기`의 경우 소제목을 `기타`에 포함시켰습니다.)
* 눈으로 보다가 확인한 오류들은 일부 수정하였습니다.
  * 예. `서초동 - 교대역 - 만경생태양재동 - 양재역 - 원산생태`, `여의도도 ㅇ- 여의도력 - 햇살도시락` 등

## 검색

* 식당 검색에는 [Kakao 지도/로컬 API](https://developers.kakao.com/features/platform#%EC%A7%80%EB%8F%84-%EB%A1%9C%EC%BB%AC)를 사용하였습니다.
  * 검색 시 음식점 카테고리(FD6)로 검색하였습니다.
  * `동 이름`과 `음식점 이름`으로 검색하였으며 `음식점 이름`에 괄호가 있는 경우 괄호 및 괄호 안의 내용은 무시합니다.
  * 검색 결과의 첫번째 값을 사용하였으며 없는 경우에는 무시합니다.
* [GeoJSON](https://tools.ietf.org/html/rfc7946) 형태로 정리하였습니다.

## 결과


In [1]:
import re
import json
import requests

In [2]:
with open("raw/raw_01-45.json", "r", encoding="utf-8") as raw_data:
    matzip_raw = json.load(raw_data)

In [3]:
url = "http://dapi.kakao.com/v2/local/search/keyword.json"

params = {
    "category_group_code":"FD6",
    "query":""
}

headers = {
    'Authorization': "KakaoAK ###_KAKAO_API_KEY_###",
}

In [4]:
geoitem_proto = {
  "type": "Feature",
  "geometry": {
    "type": "Point",
    "coordinates": [0, 0]
  },
  "properties": {
    "name": "",
    "address": "",
    "website": ""
  }
}

categories = matzip_raw.keys()

ex_ptrn = re.compile('\(.*?\)')

In [5]:
for cate in categories:
    filename = "geo/{}.geojson".format(cate.replace(' ', ''))
    is_first = True
    
    with open(filename, "w", encoding="utf-8") as json_file:
        print("Writing '{}'...".format(filename))
        json_file.write("{\"type\": \"FeatureCollection\",\"features\": [\n")

        for matzip in matzip_raw[cate]:
            items = matzip.split(" - ")

            if len(items) == 1:
                continue
            elif len(items) == 2:
                query = items[0] +" "+ re.sub(ex_ptrn, "", items[1])
            elif len(items) >= 3:
                query = items[0] +" "+ re.sub(ex_ptrn, "", items[2])

            params['query'] = query
            response = requests.request("GET", url, headers=headers, params=params)

            data = response.json()

            geoitem = geoitem_proto
            if len(data['documents']) >= 1:
                geoitem['geometry']['coordinates'] = [float(data['documents'][0]['x']), float(data['documents'][0]['y'])]
                geoitem['properties']['name'] = data['documents'][0]['place_name']
                geoitem['properties']['source'] = matzip
                geoitem['properties']['address'] = data['documents'][0]['road_address_name']
                geoitem['properties']['website'] = data['documents'][0]['place_url']

            if is_first:
                is_first = False
                json_file.write("  ")
            else:
                json_file.write("  ,")

            json_file.write(json.dumps(geoitem) + "\n")

        json_file.write("]}\n")

print("Done")

Writing 'geo/평양냉면.geojson'...
Writing 'geo/메밀국수(소바).geojson'...
Writing 'geo/막국수.geojson'...
Writing 'geo/콩국수.geojson'...
Writing 'geo/국밥,해장국.geojson'...
Writing 'geo/설렁탕.geojson'...
Writing 'geo/감자탕.geojson'...
Writing 'geo/순대.geojson'...
Writing 'geo/닭볶음탕.geojson'...
Writing 'geo/추어탕.geojson'...
Writing 'geo/육개장.geojson'...
Writing 'geo/대구탕.geojson'...
Writing 'geo/김밥.geojson'...
Writing 'geo/김치찌개.geojson'...
Writing 'geo/부대찌개.geojson'...
Writing 'geo/청국장.geojson'...
Writing 'geo/된장.geojson'...
Writing 'geo/간장게장.geojson'...
Writing 'geo/삼계탕.geojson'...
Writing 'geo/보쌈.geojson'...
Writing 'geo/족발.geojson'...
Writing 'geo/치킨.geojson'...
Writing 'geo/돈까스.geojson'...
Writing 'geo/함박스테이크.geojson'...
Writing 'geo/떡볶이.geojson'...
Writing 'geo/라면.geojson'...
Writing 'geo/라멘.geojson'...
Writing 'geo/우동.geojson'...
Writing 'geo/튀김.geojson'...
Writing 'geo/순두부,두부.geojson'...
Writing 'geo/피자.geojson'...
Writing 'geo/아이스크림,젤라또.geojson'...
Writing 'geo/갓포요리집.geojson'...
Writing 'geo/죽.geojson'...
